In [ ]:
#!pip install scikit-optimize

In [1]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import scipy
from string import punctuation
from wordcloud import STOPWORDS
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt.plots import plot_convergence,plot_objective, plot_evaluations
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.ensemble import VotingClassifier
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import torch
import pandas as pd
import torchtext
from torchtext import data
import spacy
import random
from torchtext.vocab import GloVe
import torch.nn as nn

C:\Users\mmonn\Anaconda3\envs\pytorch-env\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
C:\Users\mmonn\Anaconda3\envs\pytorch-env\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


# Raw Data Preprocess : from raw Kaggle data to combined  clean DataFrame format.

## Merge Train set.

In [2]:
#titre text entrainement
data_text = pd.read_csv('Data/text.csv')
len(data_text)

25561

In [3]:
#label associe aux titre entrainement
data_train_label = pd.read_csv('Data/train.csv')
data_train_label.head()
len(data_train_label['id'].unique())

12779

In [4]:
#table de reference
data_ref = pd.read_csv('Data/reference.csv')

In [5]:
data_test = pd.read_csv('Data/test.csv')
data_test.head()

,id
0,1
1,2
2,4
3,5
4,7


In [6]:
#Using this functions to associate each text to their IDs, each reference to their text, and each reference to their own text.
#Then i concat my titles and their reference, and separate them with /.
#Title separated from reference with /
#References are split on dot. So . separates two references of the same text.
def data_preprocess(x,y,ref):
    def ref_func(i,ref_df):
        ref_list = list(ref_df[ref_df.id_x==i]["title"])
        return ". ".join(ref_list)
    df_tmp = y.merge(x,left_on="id", right_on="id",how="inner") # Merge the label and text into on df
    ref_merge = ref.merge(x,left_on="id.1",right_on="id",how="left")
    df_tmp["new_var"] = df_tmp.id.apply(lambda i: ref_func(i,ref_merge))
    df_tmp["AllCombined"] = df_tmp["title"] + " / " + df_tmp["new_var"]
    return df_tmp

In [7]:
df_train_final = data_preprocess(data_text, data_train_label, data_ref)

In [ ]:
df_train_final.to_csv('data_train_final.csv', index = None)

In [ ]:
#Final DataFrame for training set.
df = pd.read_csv('Data/data_train_final.csv')

In [8]:
df_train_final

,id,label,title,new_var,AllCombined
0,0,1,interactive visual exploration of neighbor bas...,a framework for clustering evolving data strea...,interactive visual exploration of neighbor bas...
1,3,1,relational division four algorithms and their ...,implementation techniques for main memory data...,relational division four algorithms and their ...
2,6,1,simplifying xml schema effortless handling of ...,statix making xml count. answering xml queries...,simplifying xml schema effortless handling of ...
3,8,0,funbase a function based information managemen...,temporal databases status and research direc...,funbase a function based information managemen...
4,9,0,inverted matrix efficient discovery of frequen...,dynamic itemset counting and implication rules...,inverted matrix efficient discovery of frequen...
...,...,...,...,...,...
12774,25547,4,scaling up from dialogue to multilogue some pr...,resolving ellipsis in clarification,scaling up from dialogue to multilogue some pr...
12775,25548,3,a laboratory for the development and evaluatio...,,a laboratory for the development and evaluatio...
12776,25554,2,an analysis of transformational analogy genera...,,an analysis of transformational analogy genera...
12777,25555,2,exploiting known taxonomies in learning overla...,hierarchical classification combining bayes wi...,exploiting known taxonomies in learning overla...


## Merge Test set.

In [9]:
data_ref = pd.read_csv('Data/reference.csv')
len(data_ref)

73313

In [10]:
data_test = pd.read_csv('Data/test.csv')
len(data_test)

12782

In [11]:
data_text = pd.read_csv('Data/text.csv')
len(data_text)

25561

In [12]:
def data_preprocess(x,y,ref):
    def ref_func(i,ref_df):
        ref_list = list(ref_df[ref_df.id_x==i]["title"])
        return ". ".join(ref_list)
    df_tmp = y.merge(x,left_on="id", right_on="id",how="inner") # Merge the label and text into on df
    ref_merge = ref.merge(x,left_on="id.1",right_on="id",how="left")
    df_tmp["new_var"] = df_tmp.id.apply(lambda i: ref_func(i,ref_merge))
    df_tmp["AllCombined"] = df_tmp["title"] + " / " + df_tmp["new_var"]
    return df_tmp

In [13]:
#test_process is my final DataFrame clean and combined on which i will perform my Kaggle Predictions.
test_process = data_preprocess(x = data_text, y = data_test, ref = data_ref)

# Best  Kaggle Model: VotingClassifier with LGBM + MultinomialNB + LinearSVC (Machine Learning).

Here i created a VotingClassifier based on three very different learners in order to capture as much feature mapping as possible.
> This VotingClassifier gave me my ***best Kaggle performance and is my final model used for the competition***. Scored 83.39% on kaggle board during competition.

The base learners used were:

 1) LinearSVM (optimized with Bayesian Optimization for strongest performing base learner, ***see code at end of section***). 82.24% validation accuracy, 82.92% kaggle accuracy. Best SINGLE model.

 2) MultinomialNB (optimized with Bayesian Optimization for strongest performing base learner, ***see code at end of section***).
82.001% validation accuracy. Did not submit this model alone on kaggle (LinearSVM was better)

 3) LGBMClassifier (optimized with Bayesian Optimization for strongest performing base learner, ***see code at end of section***).
79.39% validation accuracy. Very bad model alone BUT works VERY well in VotingClassifier. Must have interesting feature mapping.

In [44]:
df = pd.read_csv('Data/data_train_final.csv')

In [45]:
stemmer = PorterStemmer()

In [46]:
nlp = spacy.load('en_core_web_lg')

In [47]:
# to clean data (taking out special characters and multiple spaces)
df['new_AllCombined'] = df['AllCombined'].apply(lambda x: re.sub(r"[^A-Za-z0-9()!?\'\`\"]", " ",x))  
df['new_AllCombined'] = df['new_AllCombined'].apply(lambda x: re.sub("\s{2,}", " ",x)) 

In [48]:
#df['new_AllCombined'] = df['new_AllCombined'].apply(lambda x: [stemmer.stem(y) for y in x.split()])

In [49]:
#stemming or lemmatization both need this.
#df['new_AllCombined'] = df['new_AllCombined'].apply(lambda x : ' '.join(x) )

In [50]:
#take out all non alpha numeric
df['new_AllCombined'] = df.new_AllCombined.apply(lambda x: re.sub(r'[^\w\s]','', x))

## Preprocess for LogReg / SVC (to be passed in TfidfVectorizer for preprocessing). STEMMING FUNCT.

Certain models got stemming, others lemmatization, others none. I based my decision purely on accuracy. I will not show every single test i made for every single model or this will be too long.

In [51]:
#PREPROCESSING FUNCTION FOR VOTINGCLASSIFIER
def my_preprocess(text):
    text = re.sub(r"[^A-Za-z0-9()!?\'\`\"]", " ",text)
    text = re.sub("\s{2,}", " ",text)
    text = re.sub(r'[^\w\s]','', text)
    # text = text.apply(lambda x: re.sub(r"[^A-Za-z0-9()!?\'\`\"]", " ",x)) --> peut pas utiliser de lambda function ici.
    # text = text.apply(lambda x: re.sub("\s{2,}", " ",x))
    # text = text.apply(lambda x: re.sub(r'[^\w\s]','', x))
    return text

In [52]:
#PREPROCESSING FUNCTION FOR VOTINGCLASSIFIER
tokenizer = Tokenizer(nlp.vocab)
def my_tokenizer(doc):
    # tokens = doc.split()
    tokens = [token.text for token in tokenizer(doc)]
    tokens = [stemmer.stem(x) for x in tokens]  # --> l'argument tokenizer de TfidfVectorizer prend une list de tokens, cest pour ca qu'on fait ca ici
    return tokens

## Preprocess for NB / LGBM (to be passed in TfidfVectorizer for preprocessing). LEMMATIZATION FUNCT.

In [53]:
#PREPROCESSING FUNCTION FOR VOTINGCLASSIFIER
def my_preprocess1(text):
    text = re.sub(r"[^A-Za-z0-9()!?\'\`\"]", " ",text)
    text = re.sub("\s{2,}", " ",text)
    text = re.sub(r'[^\w\s]','', text)
    return text

In [54]:
#lemmatizing moin bon resultats aussi, mais reste du preprocess augmente results
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [55]:
#PREPROCESSING FUNCTION FOR VOTINGCLASSIFIER
tokenizer = Tokenizer(nlp.vocab)
def my_tokenizer1(doc):
    def lemmatize_text(tok):
        return lemmatizer.lemmatize(tok)
    tokens = [token.text for token in tokenizer(doc)]
    tokens = [lemmatize_text(x) for x in tokens]  # --> l'argument tokenizer de TfidfVectorizer prend une list de tokens, cest pour ca qu'on fait ca ici
    return tokens

In [56]:
X_train, X_test, y_train, y_test = train_test_split(df['new_AllCombined'], df['label'], stratify = df['label'], shuffle = True, random_state = 42, test_size = 0.3)

# Pipeline

In [57]:
#best model pipeline/params with 82.24% acc solo
#LinearSVC + stemming
text_clf_svc = Pipeline([('tfidf', TfidfVectorizer(ngram_range = (1,6),
                                                stop_words = None,
                                                preprocessor = my_preprocess,
                                                tokenizer = my_tokenizer)),
                        
                         ('clf', LinearSVC(C=1.0069937972047027,
                                           loss = 'hinge',
                                           tol = 0.5,
                                           max_iter = 100000000, 
                                           fit_intercept=False, 
                                           random_state=42))])

#pour train
# text_clf.fit(X_train, y_train)

In [58]:
#82% solo
#MultinomialNB + lemmatization
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,6), 
                                               stop_words=STOP_WORDS, 
                                               norm = 'l1', 
                                               strip_accents=None,
                                               lowercase = True,
                                               analyzer = 'word',
                                               preprocessor = my_preprocess1,
                                               tokenizer = my_tokenizer1)),
                     
                    ('nb_model', MultinomialNB(alpha = 0.01852770799933675,
                                           fit_prior = False))])

In [59]:
#text_clf_lr = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,4), 
#                                                  stop_words='english',
#                                                  preprocessor = my_preprocess,
#                                                  tokenizer = my_tokenizer)), 
#                        
#                        ('lr', LogisticRegression(C=4.5,multi_class='ovr',random_state=42, max_iter=100000))])

In [60]:
##skopt model. Version Lemmatization + tree.
#text_clf_lgb = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,1),
#                                  stop_words = None,
#                                  norm = 'l2',
#                                  tokenizer = my_tokenizer1, 
#                                  preprocessor = my_preprocess1)),
#                         
#                    ('lgb', LGBMClassifier(importance_type = 'split',
#                                           boosting_type = 'gbdt',
#                                           learning_rate = 0.044216356170560425,
#                                           n_estimators = 820,
#                                           subsample = 0.6380506568792133,
#                                           colsample_bytree = 0.9154582072806454,
#                                           reg_alpha = 0.2045409147444816,
#                                           reg_lambda = 0.7049387326724629,
#                                           random_state = 42,
#                                           n_jobs = -1))])

In [61]:
#skopt model. Version Lemmatization + dart. (best model mix) - 79.37% solo
#LGBMClassifier + lemmatization.
text_clf_lgb = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,1),
                                  stop_words = None,
                                  norm = 'l2',
                                  tokenizer = my_tokenizer1, 
                                  preprocessor = my_preprocess1)),
                         
                    ('lgb', LGBMClassifier(importance_type = 'split',
                                           boosting_type = 'dart',
                                           learning_rate = 0.058067079891947884,
                                           n_estimators = 1903,
                                           subsample = 0.6941823248699718,
                                           colsample_bytree = 0.6649289756607163,
                                           reg_alpha = 0.3617243544568278,
                                           reg_lambda = 1.354455342583696,
                                           random_state = 42,
                                           n_jobs = -1))])

In [62]:
##benchmark (avec stopwords et stemming)
#text_clf_lgb = Pipeline([('tfidf', TfidfVectorizer(stop_words = STOP_WORDS,
#                                                   tokenizer=my_tokenizer, 
#                                                   preprocessor=my_preprocess)),
#                         
#                         ('lgb',LGBMClassifier(random_state = 42, 
#                                               max_iter = 10000))])

In [63]:
#RandomForstClassifier
#text_clf_rf = Pipeline([('tfidf', TfidfVectorizer(ngram_range = (1,2),
#                                                   tokenizer=my_tokenizer, 
#                                                   preprocessor=my_preprocess)),
#                         
#                         ('lgb',RandomForestClassifier(random_state = 42, 
#                                               ))])

In [64]:
#BEST VOTING CLASSIFIER. ALSO BEST KAGGLE MODEL. 82.76% valid accuracy, 83.39% Kaggle Accuracy.
#82.76% avec new tweaked lgb + stemming + dart
voting_clf8 = VotingClassifier(
    estimators=[('nb', text_clf_nb), ('lgb', text_clf_lgb),('svc', text_clf_svc)],
    voting = 'hard',
    weights = [1,1,1]
    
)

In [65]:
#82.13%
#voting_clf1 = VotingClassifier(
#    estimators=[('lr',text_clf_lr), ('nb', text_clf_nb), ('svc', text_clf_svc)],
#    voting = 'hard'
#)

In [66]:
#82.42%
#voting_clf2 = VotingClassifier(
#    estimators=[('nb', text_clf_nb), ('svc', text_clf_svc)],
#    voting = 'hard'
#)

In [67]:
#81,71%
#voting_clf3 = VotingClassifier(
#    estimators=[('lr',text_clf_lr), ('nb', text_clf_nb)],
#    voting = 'soft'
#)

In [68]:
# 81,11%
#voting_clf4 = VotingClassifier(
#    estimators=[('lr',text_clf_lr), ('nb', text_clf_nb), ('lgb', text_clf_lgb)],
#    voting = 'soft'
#)

In [69]:
# 81,89%%
#voting_clf5 = VotingClassifier(
#    estimators=[('lr',text_clf_lr), ('nb', text_clf_nb), ('lgb', text_clf_lgb)],
#    voting = 'hard'
#)

In [70]:
#82.47%
#voting_clf6 = VotingClassifier(
#    estimators=[('lr',text_clf_lr), ('nb', text_clf_nb), ('lgb', text_clf_lgb),('svc', text_clf_svc)],
#    voting = 'hard',
#    
#)

In [71]:
#82.42
#voting_clf7 = VotingClassifier(
#    estimators=[('lr',text_clf_lr), ('nb', text_clf_nb), ('lgb', text_clf_lgb),('svc', text_clf_svc)],
#    voting = 'hard',
#    weights = [1,1.1,0.9,1.2]
#)

In [72]:
#82.23%
#voting_clf9 = VotingClassifier(
#    estimators=[('lr',text_clf_lr), ('nb', text_clf_nb), ('lgb', text_clf_lgb),('svc', text_clf_svc),('rf', text_clf_rf)],
#    voting = 'hard',
#    
#)

In [73]:
#80.28%
#voting_clf10 = VotingClassifier(
#    estimators=[('lr',text_clf_lr), ('nb', text_clf_nb), ('lgb', text_clf_lgb),('rf', text_clf_rf)],
#    voting = 'soft'
#
#)

In [74]:
#FOR Training (X_train, y_train)
voting_clf8.fit(X_train, y_train)

C:\Users\mmonn\Anaconda3\envs\pytorch-env\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [' ', 'd', 'doe', 'ha', 'le', 'll', 'm', 'n', 'nt', 's', 't', 'u', 've', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


VotingClassifier(estimators=[('nb',
                              Pipeline(memory=None,
                                       steps=[('tfidf',
                                               TfidfVectorizer(analyzer='word',
                                                               binary=False,
                                                               decode_error='strict',
                                                               dtype=<class 'numpy.float64'>,
                                                               encoding='utf-8',
                                                               input='content',
                                                               lowercase=True,
                                                               max_df=1.0,
                                                               max_features=None,
                                                               min_df=1,
                                                 

In [20]:
#FOR Final prediction. Fitting on WHOLE dataset instead of (X_train, y_train)
voting_clf8.fit(df['new_AllCombined'], df['label'])

C:\Users\mmonn\Anaconda3\envs\pytorch-env\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [' ', 'd', 'doe', 'ha', 'le', 'll', 'm', 'n', 'nt', 's', 't', 'u', 've', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


VotingClassifier(estimators=[('nb',
                              Pipeline(memory=None,
                                       steps=[('tfidf',
                                               TfidfVectorizer(analyzer='word',
                                                               binary=False,
                                                               decode_error='strict',
                                                               dtype=<class 'numpy.float64'>,
                                                               encoding='utf-8',
                                                               input='content',
                                                               lowercase=True,
                                                               max_df=1.0,
                                                               max_features=None,
                                                               min_df=1,
                                                 

In [75]:
#Best model results at 82.759%
voting_clf8.score(X_test, y_test)

0.8275952008346374

In [21]:
data_text = pd.read_csv('Data/text.csv')
len(data_text)

25561

In [22]:
data_test = pd.read_csv('Data/test.csv')
len(data_test)

12782

In [23]:
data_ref = pd.read_csv('Data/reference.csv')
len(data_ref)

73313

In [24]:
#utilisation fonction pour faire le merging avec references
def data_preprocess(x,y,ref):
    def ref_func(i,ref_df):
        ref_list = list(ref_df[ref_df.id_x==i]["title"])
        return ". ".join(ref_list)
    df_tmp = y.merge(x,left_on="id", right_on="id",how="inner") # Merge the label and text into on df
    ref_merge = ref.merge(x,left_on="id.1",right_on="id",how="left")
    df_tmp["new_var"] = df_tmp.id.apply(lambda i: ref_func(i,ref_merge))
    df_tmp["AllCombined"] = df_tmp["title"] + " / " + df_tmp["new_var"]
    return df_tmp

In [25]:
test_process = data_preprocess(x = data_text, y = data_test, ref = data_ref)

In [26]:
test_process

,id,title,new_var,AllCombined
0,1,autodomainmine a graphical data mining system ...,what is the nearest neighbor in high dimension...,autodomainmine a graphical data mining system ...
1,2,anipqo almost non intrusive parametric query o...,parametric query optimization. optimization of...,anipqo almost non intrusive parametric query o...
2,4,selection and ranking of text from highly impe...,,selection and ranking of text from highly impe...
3,5,conditional random fields for multi agent rein...,sequential optimality and coordination in mult...,conditional random fields for multi agent rein...
4,7,multi dimensional description logics,,multi dimensional description logics /
...,...,...,...,...
12777,25553,currency based updates to distributed material...,database snapshots. maintenance of views. impl...,currency based updates to distributed material...
12778,25556,dynamic typing in a statically typed language,quasi static typing. an ideal model for recurs...,dynamic typing in a statically typed language ...
12779,25558,learning sparse metrics via linear programming,fastmap a fast algorithm for indexing data min...,learning sparse metrics via linear programming...
12780,25559,computer assisted reasoning with mizar,,computer assisted reasoning with mizar /


In [27]:
# to clean data (taking out special characters and multiple spaces)
test_process['new_AllCombined'] = test_process['AllCombined'].apply(lambda x: re.sub(r"[^A-Za-z0-9()!?\'\`\"]", " ",x))  
test_process['new_AllCombined'] = test_process['new_AllCombined'].apply(lambda x: re.sub("\s{2,}", " ",x))

In [28]:
#take out all non alpha numeric
test_process['new_AllCombined'] = test_process.new_AllCombined.apply(lambda x: re.sub(r'[^\w\s]','', x))

In [29]:
#This is my final prediction to kaggle. (voting_clf8 was fitted on the WHOLE dataset before final preds.)
y_pred = voting_clf8.predict(test_process.new_AllCombined)

In [30]:
y_pred_df = pd.DataFrame(y_pred, columns=['label'])

In [31]:
y_pred_df.head()

,label
0,0
1,1
2,0
3,2
4,2


In [32]:
y_id_df = pd.DataFrame(test_process.id, columns = ['id'])
y_id_df

,id
0,1
1,2
2,4
3,5
4,7
...,...
12777,25553
12778,25556
12779,25558
12780,25559


In [33]:
final = pd.concat([y_id_df, y_pred_df], axis = 1) 

In [34]:
final.head()

,id,label
0,1,0
1,2,1
2,4,0
3,5,2
4,7,2


In [35]:
#BEST PREDICTIONS FOR KAGGLE IN THIS CSV. 83.39%.
final.to_csv('VOTING_FINAL.csv', index = None)

In [36]:
fin = pd.read_csv('VOTING_FINAL.csv')

In [37]:
fin.head()

,id,label
0,1,0
1,2,1
2,4,0
3,5,2
4,7,2


## Bayesian Optimization for each Base Learner.

Each base learning was trained for minimum 500 iterations. (LGBMClassifier 500 iteration, MultinomialNB 1500 iterations, LinearSVC 2500 iterations). What you see plugged in is the warmstart of my final best hyper-parameters for each model.

### Bayesian LGBM : LGBMClassifier SKopt

In [ ]:
def result(func):
    def wrapper(*args,**kwargs):
        res = func(*args,**kwargs)
        print(*args,**kwargs)
        return res
    return wrapper

In [ ]:
# Define hyperparameters 

importance_type = Categorical(('split','gain'), name = 'importance_type')
#boosting_type = Categorical(('gbdt', 'dart'), name = 'boosting_type')
learning_rate = Real(low = 0.001, high = 0.2001, name = 'learning_rate')
#max_depth = Integer(low = 1, high = 15, name = 'max_depth')
n_estimators = Integer(low = 20, high = 3000, name = 'n_estimators')
subsample = Real(low = 0.5, high = 1, name = 'subsample')
colsample_bytree = Real(low = 0.5, high = 1, name = 'colsample_bytree')
reg_alpha = Real(low = 0, high = 15, name = 'reg_alpha')
reg_lambda = Real(low = 0, high = 15, name = 'reg_lambda')
#ngram_range_sup = Integer(low=1, high=3, name='ngram_range_sup')
norm = Categorical(('l1','l2'), name = 'norm')
#stop_words = Categorical((None, 'english'), name = 'stop_words')

dimensions = [importance_type, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, norm]

nb_calls = 350

default_parameters = ['split', 0.058067079891947884, 1903, 0.6941823248699718, 0.6649289756607163, 0.3617243544568278, 1.354455342583696, 'l2']

In [ ]:
@result
@use_named_args(dimensions=dimensions)
def fitness(importance_type, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, norm):
    
    text_clf = Pipeline([
        ('tfidf', TfidfVectorizer(#ngram_range=(1,ngram_range_sup),
                                  #stop_words = stop_words,
                                  norm = norm,
                                  tokenizer = my_tokenizer1, 
                                  preprocessor = my_preprocess1
                                  )),
                         
            ('lgb', LGBMClassifier(importance_type = importance_type,
                                   boosting_type = 'dart',
                                   learning_rate = learning_rate,
                                   #max_depth = max_depth,
                                   n_estimators = n_estimators,
                                   subsample = subsample,
                                   colsample_bytree = colsample_bytree,
                                   reg_alpha = reg_alpha,
                                   reg_lambda = reg_lambda,
                                   random_state = 42,
                                   n_jobs = -1
                                  ))])
    
    
    text_clf.fit(X_train, y_train)
    accurracy = text_clf.score(X_test, y_test)
    return -accurracy

if __name__ == '__main__':
    search_result = gp_minimize(func=fitness,
                                dimensions=dimensions,
                                acq_func='EI',
                                n_calls=nb_calls,
                                x0=default_parameters,
                                n_jobs=-1,
                                verbose=True,
                                random_state=4
                               )
    
print(search_result.x)
print(search_result.fun)
plot = plot_convergence(search_result)

### Baysian LinearSVM : LinearSVM Skopt

In [ ]:
def result(func):
    def wrapper(*args,**kwargs):
        res = func(*args,**kwargs)
        print(*args,**kwargs)
        return res
    return wrapper

In [ ]:
# Define hyperparameters 

# multi_class = Categorical(('auto', 'ovr','multinomial'), name = 'multi_class')
#dual = Categorical((True, False), name = 'dual')
C = Real(low=0.01, high=90, name= 'C')
ngram_range_sup = Integer(low=2, high=10, name='ngram_range_sup')
loss = Categorical(('hinge', 'squared_hinge'), name = 'loss')
stop_words = Categorical(('english', None), name = 'stop_words')
norm = Categorical(('l1','l2'), name = 'norm')
#fit_intercept = Categorical((True,False), name = 'fit_intercept')
tol = Real(low=0.00001, high = 25, name = 'tol')







dimensions = [C, ngram_range_sup, loss, stop_words, norm, tol]


nb_calls = 500

default_parameters = [1.0069937972047027, 6, 'hinge', None, 'l2', 0.5]

In [ ]:
@result
@use_named_args(dimensions=dimensions)
def fitness(C, ngram_range_sup, loss, stop_words, norm, tol):
    text_clf = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,ngram_range_sup), 
                                                   norm = norm, 
                                                   stop_words = stop_words,
                                                   tokenizer = my_tokenizer,
                                                   preprocessor = my_preprocess)),
                         
                         
                          ('clf', 
                           LinearSVC(C=C, 
                                     max_iter=10000, 
                                     loss=loss,  
                                     fit_intercept=False, 
                                     tol = tol, 
                                     random_state=42))])
    
    
    text_clf.fit(X_train, y_train)
    accurracy = text_clf.score(X_test, y_test)
    return -accurracy

if __name__ == '__main__':
    search_result = gp_minimize(func=fitness,
                                dimensions=dimensions,
                                acq_func='EI',
                                n_calls=nb_calls,
                                x0=default_parameters,
                                n_jobs=-1,
                                verbose=True,
                                random_state=4,
                                xi=0.001
                               )
    
print(search_result.x)
print(search_result.fun)
plot = plot_convergence(search_result)

### Bayesian NB : MultinomialNB Skopt

In [1]:
def result(func):
    def wrapper(*args,**kwargs):
        res = func(*args,**kwargs)
        print(*args,**kwargs)
        return res
    return wrapper

In [ ]:
# Define hyperparameters 

alpha = Real(low=0.00001, high=12, name= 'alpha')
ngram_range_sup = Integer(low=2, high=22, name='ngram_range_sup')
fit_prior = Categorical((False, True), name = 'fit_prior')
stop_words = Categorical(('english', None), name = 'stop_words')
norm = Categorical(('l1', 'l2'), name = 'norm')
strip_accents = Categorical((None, 'unicode'), name = 'strip_accents')
lowercase = Categorical((True, False), name = 'lowercase')
analyzer = Categorical(('word', 'char'), name = 'analyzer')
#essayer avec Tokenizer = 'spacy' aussi, mais marche juste si analyzer = 'word'


dimensions = [alpha, ngram_range_sup, fit_prior, stop_words, norm, strip_accents, lowercase, analyzer]


nb_calls = 800

#warmstart with best params to date. 81.95% best accuracy with MultiNB et lemmatization (clean, lemmatize, join, last cleaning function)
default_parameters = [0.01852770799933675, 6, False, 'english', 'l1', None, True, 'word']

In [ ]:
@result
@use_named_args(dimensions=dimensions)
def fitness(alpha, ngram_range_sup, fit_prior, stop_words, norm, strip_accents, lowercase, analyzer):
    text_clf = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,ngram_range_sup),
                                                   stop_words = stop_words, 
                                                   norm=norm,
                                                   strip_accents = strip_accents, 
                                                   lowercase=lowercase, 
                                                   analyzer=analyzer,
                                                   tokenizer = my_tokenizer1,
                                                   preprocessor = my_preprocess1)),
                         
                    ('clf', MultinomialNB(alpha=alpha, 
                                          fit_prior=fit_prior))])
    
    text_clf.fit(X_train, y_train)
    accurracy = text_clf.score(X_test, y_test)
    return -accurracy

if __name__ == '__main__':
    search_result = gp_minimize(func=fitness,
                                dimensions=dimensions,
                                acq_func='EI',
                                n_calls=nb_calls,
                                x0=default_parameters,
                                n_jobs=-1,
                                verbose=True,
                                random_state=4
                               )
    
print(search_result.x)
print(search_result.fun)
plot = plot_convergence(search_result)

In [ ]:
#END OF PRESENTATION OF MY BEST KAGGLE MODEL. END OF MACHINE LEARNING.

# Start of Deep Learning Part.

Best Deep Learning model:

> CNN with FastText embedding.

Deep Learning Models tested:

1) Vanilla RNN (60% validation accuracy, not tested on kaggle submission)

2) Vanilla LSTM (72% validation accuracy, not tested on kaggle submission)

3) Bi-directional, multi-layer LSTM (with GloVe 840B embedding) (81.1% validation accuracy, 81% test kaggle accuracy)

4) Bi-directional, multi-layer LSTM with BERT (81.5% validation accuracy, 81% test kaggle accuracy)

5) CNN with GloVe embedding + Conv1D and Conv2D choice models. (83% validation accuracy, 81.6% test kaggle accuracy, i think i have either overfitting problem or a problem with my prediction function with this model. In theory it should be my best model of all, but i have problems when submitting to kaggle.)

> Note: I will not post my Vanilla RNN and Vanilla LSTM as they were used to build my best LSTM model.

## Best LSTM : Bi-directional, GloVe 840B, Multi-Layer, double dropout.

In [76]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda')

In [77]:
#include_lengths = True because we are using packed padded sequences
TEXT = data.Field(sequential = True, 
                  lower = True, 
                  use_vocab = True,
                  tokenize = 'spacy', 
                  include_lengths = True, 
                  batch_first=True)

LABEL = data.LabelField(sequential = False, use_vocab = False)


In [78]:
#defining tuples (le premier du tuple doit match notre nom de dataset tabulaire, le deuxieme doit match son field)
fields = [("id",None),("label",LABEL),("title",None),("new_var",None),("AllCombined",TEXT)]

In [79]:
#importer les donnees dependement de si on a train, valid, test ou bien juste train / test. Ici on a juste un fichier train, voir sentiment 1 pour multi fichier options (pas de .split apres TabularDataset)
train = data.TabularDataset(path = 'Data/data_train_final.csv',
                                format = 'csv',
                                fields = fields,
                                skip_header = True )

In [80]:
#split en train et valid
train_data, valid_data = train.split(split_ratio = 0.8, random_state = random.seed(SEED))
print(f'{(len(train_data))} {len(valid_data)}')

10223 2556


In [81]:
TEXT.build_vocab(train_data, 
                 vectors = "glove.840B.300d", 
                 #glove.840B.300d
                 #glove.6B.100d
                 unk_init = torch.Tensor.normal_)

# Find the padding index and the vocab size
padding_idx = TEXT.vocab.stoi["<pad>"]
vocab_size = len(TEXT.vocab)
vocab_size

9971

In [82]:
#create iterators after defining batch size and device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#creation de l'iterateur
(train_iter, valid_iter) = data.BucketIterator.splits((train_data, valid_data), 
                                                    batch_sizes = (32,32), #batch_size pour un argument, batch_sizes pour tuple
                                                    device = device,
                                                    sort_key = lambda x: len(x.AllCombined),
                                                    sort_within_batch = True
                                                   #sort_within_batch sert pour le padding
                                                   )

In [83]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        #rajoute padding_idx = pad_idx pour dire au model comment skip le padding.
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        #defini le lstm
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers = n_layers,
                           bidirectional = bidirectional,
                           dropout = 0 if n_layers < 2 else dropout,
                           batch_first = True)
        
        #defini le linear (le * 2 viens du fait qu'on va utiliser bidirectional)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        #defini le dropout
        self.dropout = nn.Dropout(dropout)
        
    #As we are passing the lengths of our sentences to be able to use packed padded sequences, 
    #we have to add a second argument, `text_lengths`, to `forward`.    
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        #comme on a pas fait batch_first = True, notre batch size est pas en premier dans le tensor
        
        #pack sequence:
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first = True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        #unpack sequence:
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        
         #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
        #hidden = [batch size, hid dim * num directions]
        
        return self.fc(hidden)
        
        

In [84]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 250
OUTPUT_DIM = 5
N_LAYERS = 4
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [85]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,609,805 trainable parameters


In [86]:
#copying pre-trained word embeddings
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([9971, 300])


In [87]:
#replace initial weights
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.3195,  0.2435, -0.1777,  ..., -0.0243, -0.6011,  0.2368],
        [-0.0907, -0.1088,  1.1903,  ..., -1.1861,  1.3736, -0.4054]])

In [88]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.3195,  0.2435, -0.1777,  ..., -0.0243, -0.6011,  0.2368],
        [-0.0907, -0.1088,  1.1903,  ..., -1.1861,  1.3736, -0.4054]])


In [89]:
import torch.optim as optim

optimizer = optim.RMSprop(model.parameters(), lr = 1e-5)

In [90]:
criterion = nn.CrossEntropyLoss()

In [91]:
#model we instanciated earlier, pass it to GPU.
model = model.to(device)
criterion = criterion.to(device)

In [92]:
# trainning loop (modified version of homework1 trainning func)
def train(model, iterator, optimizer, criterion):
    total_loss, total_correct, total_prediction = 0.0, 0.0, 0.0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.AllCombined # IMPORTANT : ligne a ajouter quand on PACK. Utiliser notre field de text dans batch. On fait ca a cause du packing
        logits = model(text.cuda(), text_lengths) #comme on a unpack au dessus, no motre batch. comme dans le rnn
        
        predictions = torch.max(logits, dim=-1)[1]
        loss = criterion(logits, batch.label.cuda()) #ici a la place de label faut mettre le nom de notre y du tabular dataset initial ('label' pour nous)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_correct += torch.eq(predictions, batch.label.cuda()).sum().item()
        total_prediction += batch.label.size(0)
    return total_loss / len(iterator), total_correct / total_prediction

In [93]:
# evaluation loop (modified version of homework1 eval func)
def evaluate(model, iterator, criterion):  
    total_loss, total_correct, total_prediction = 0.0, 0.0, 0.0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.AllCombined #IMPORTANT, ajouter cette ligne quand PACK.utiliser notre field de text dans batch. On fait ca a cause du packing
            logits = model(text.cuda(), text_lengths) #comme on a unpack au dessus, no motre batch. comme dans le rnn
            predictions = torch.max(logits, dim=-1)[1]
            loss = criterion(logits, batch.label.cuda())

            total_loss += loss.item()
            total_correct += torch.eq(predictions, batch.label.cuda()).sum().item()
            total_prediction += batch.label.size(0)
    return total_loss / len(iterator), total_correct / total_prediction

In [94]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [95]:
import os
import copy

In [ ]:
N_EPOCHS = 10

best_valid_acc = 0.0
best_epoch = 1

# model 2 100 x 300, 4 layers, bidirectional --> 81.2
PATH = f"LSTMrmsp1000.pt"
if os.path.exists(PATH):
    print("Loading model from last checkpoint...")
    state = torch.load(PATH)
    model.load_state_dict(state['best_state_dict'])
    best_valid_acc = state['best_valid_acc']
    best_epoch = state['epoch']
    has_checkpoint = True
print(f"Best Validations Accuracy so far: {best_valid_acc:.3f} at Epoch {best_epoch}\n")


for epoch in range(N_EPOCHS):
    
    real_epoch = best_epoch + epoch

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion) #rentrer train_iter et valid_iter qu'on a defini dans notre BucketIterator lors du preprocess
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_state_dict = copy.deepcopy(model.state_dict())
        print(f'Epoch {real_epoch: <{5}} | Epoch Time: {epoch_mins}m {epoch_secs}s \n\
        | Train loss {train_loss:8.3f}| Train acc {train_acc:8.3f} | Valid loss {valid_loss:8.3f} | Valid acc {valid_acc:8.3f} | + ')
        #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        
        if not os.path.exists('Models'):
            os.makedirs('Models')
                
            # Let's create the checkpoint data to save
        checkpoint = {
            'epoch': real_epoch,
            'best_valid_acc': best_valid_acc,
            'best_state_dict': best_state_dict,
        }
        torch.save(checkpoint, PATH)
    else:
        print(f'Epoch {real_epoch: <{5}} | Epoch Time: {epoch_mins}m {epoch_secs}s \n\
              |  Train loss {train_loss:8.3f}| Train acc {train_acc:8.3f} | Valid loss {valid_loss:8.3f} | Valid acc {valid_acc:8.3f} |')


Best Validations Accuracy so far: 0.000 at Epoch 1

Epoch 1     | Epoch Time: 0m 19s 
        | Train loss    0.848| Train acc    0.783 | Valid loss    0.609 | Valid acc    0.795 | + 
Epoch 2     | Epoch Time: 0m 18s 
        | Train loss    0.538| Train acc    0.828 | Valid loss    0.589 | Valid acc    0.800 | + 


In [ ]:
PATH=f"LSTMrmsp1000.pt"
state = torch.load(PATH)
model.load_state_dict(state['best_state_dict'])

In [ ]:
#pred pour la soumission une fois qu'on a trouve le meilleur model
#voir detail dans Programming Pytorch for Deep Learning
def prediction(test):
    model.eval()
    processed = TEXT.process([TEXT.preprocess(test)])
    text, text_lengths = processed
    preds = model(text.cuda(), text_lengths).argmax().item()
    return preds

In [ ]:
y_pred = [[test_process.id[i], prediction(test_process.AllCombined[i])] for i in range(len(test_process))]

In [ ]:
y_pred_df = pd.DataFrame(y_pred, columns = ['id', 'label'])

In [ ]:
y_pred_df

In [ ]:
y_pred_df.to_csv('y_pred_df_rmsp1.csv', index = None)

## Best CNN Models with GloVe and choice between Conv2D model or Conv1D model.

In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

#torch.manual_seed(SEED)


TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(use_vocab = False)

In [2]:
#defining tuples (le premier du tuple doit match notre nom de dataset tabulaire, le deuxieme doit match son field)
fields = [("id",None),("label",LABEL),("title",None),("new_var",None),("AllCombined",TEXT)]

In [3]:
#importer les donnees dependement de si on a train, valid, test ou bien juste train / test. Ici on a juste un fichier train, voir sentiment 1 pour multi fichier options (pas de .split apres TabularDataset)
train = data.TabularDataset(path = 'Data/data_train_final.csv',
                                format = 'csv',
                                fields = fields,
                                skip_header = True )

In [4]:
train_data, valid_data = train.split(split_ratio = 0.8, random_state = random.seed(SEED))

In [5]:
print(f'Training has {len(train_data)} observations \nValidation has {len(valid_data)} observations')

Training has 10223 observations 
Validation has 2556 observations


In [6]:
TEXT.build_vocab(train_data, 
                 vectors = "glove.840B.300d", 
                 unk_init = torch.Tensor.normal_)

In [7]:
len(TEXT.vocab)

9971

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, valid_iter = data.BucketIterator.splits((train_data, valid_data),
                                                    batch_sizes = (50,50),
                                                    device = device,
                                                    sort_key = lambda x: x.AllCombined,
                                                    sort_within_batch = True)

### CNN with Conv2D

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
            
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1,
                                              out_channels = n_filters,
                                              kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent_len, batch_size]
        
        #text = text.permute(1,0)
        
        #text = [batch_size, sent_len]
        
        embedded = self.embedding(text)
        #embedded = [batch_size, sent_len, emb_dim]
        
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [10]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 5
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,353,105 trainable parameters


In [12]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.8215,  1.1231, -0.1510,  ..., -0.4182, -0.9061, -0.3446],
        [ 1.0604,  0.2917,  0.2861,  ...,  1.1272, -0.0987,  1.1603],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.3195,  0.2435, -0.1777,  ..., -0.0243, -0.6011,  0.2368],
        [-0.5658,  0.8013, -0.1055,  ...,  1.6674,  0.0561, -0.2571]])

In [13]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.3195,  0.2435, -0.1777,  ..., -0.0243, -0.6011,  0.2368],
        [-0.5658,  0.8013, -0.1055,  ...,  1.6674,  0.0561, -0.2571]])

In [14]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 1e-4)

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [15]:
# train a model
def train(model, iterator, optimizer, criterion):
    total_loss, total_correct, total_prediction = 0.0, 0.0, 0.0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        logits = model(batch.AllCombined.cuda()) #a la place de .text (dans batch.text) il 
                                            #faut mettre le X du tabular dataset initial ('title' pour nous)
        
        predictions = torch.max(logits, dim=-1)[1]
        loss = criterion(logits, batch.label.cuda()) #ici a la place de label faut mettre le nom de notre y du tabular dataset initial ('label' pour nous)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_correct += torch.eq(predictions, batch.label.cuda()).sum().item()
        total_prediction += batch.label.size(0)
    return total_loss / len(iterator), total_correct / total_prediction

In [16]:
# evaluate a model
def evaluate(model, iterator, criterion):  
    total_loss, total_correct, total_prediction = 0.0, 0.0, 0.0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            logits = model(batch.AllCombined.cuda()) #changer text pour title dans batch.text
            predictions = torch.max(logits, dim=-1)[1]
            loss = criterion(logits, batch.label.cuda())

            total_loss += loss.item()
            total_correct += torch.eq(predictions, batch.label.cuda()).sum().item()
            total_prediction += batch.label.size(0)
    return total_loss / len(iterator), total_correct / total_prediction

In [17]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [18]:
import os
import copy

In [19]:
N_EPOCHS = 30

best_valid_acc = 0.0
best_epoch = 1

# model 2 100 x 300, 2 layers, unidirectional --> 80.9
PATH = f"cnn12-test.pt"
if os.path.exists(PATH):
    print("Loading model from last checkpoint...")
    state = torch.load(PATH)
    model.load_state_dict(state['best_state_dict'])
    best_valid_acc = state['best_valid_acc']
    best_epoch = state['epoch']
    has_checkpoint = True
print(f"Best Validations Accuracy so far: {best_valid_acc:.3f} at Epoch {best_epoch}\n")


for epoch in range(N_EPOCHS):
    
    real_epoch = best_epoch + epoch

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion) #rentrer train_iter et valid_iter qu'on a defini dans notre BucketIterator lors du preprocess
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_state_dict = copy.deepcopy(model.state_dict())
        print(f'Epoch {real_epoch: <{5}} | Epoch Time: {epoch_mins}m {epoch_secs}s \n\
        | Train loss {train_loss:8.3f}| Train acc {train_acc:8.3f} | Valid loss {valid_loss:8.3f} | Valid acc {valid_acc:8.3f} | + ')
        #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        
        if not os.path.exists('Models'):
            os.makedirs('Models')
                
            # Let's create the checkpoint data to save
        checkpoint = {
            'epoch': real_epoch,
            'best_valid_acc': best_valid_acc,
            'best_state_dict': best_state_dict,
        }
        torch.save(checkpoint, PATH)
    else:
        print(f'Epoch {real_epoch: <{5}} | Epoch Time: {epoch_mins}m {epoch_secs}s \n\
              |  Train loss {train_loss:8.3f}| Train acc {train_acc:8.3f} | Valid loss {valid_loss:8.3f} | Valid acc {valid_acc:8.3f} |')

Best Validations Accuracy so far: 0.000 at Epoch 1

Epoch 1     | Epoch Time: 0m 8s 
        | Train loss    1.366| Train acc    0.477 | Valid loss    1.092 | Valid acc    0.710 | + 
Epoch 2     | Epoch Time: 0m 7s 
        | Train loss    0.956| Train acc    0.701 | Valid loss    0.805 | Valid acc    0.751 | + 
Epoch 3     | Epoch Time: 0m 7s 
        | Train loss    0.762| Train acc    0.749 | Valid loss    0.704 | Valid acc    0.761 | + 
Epoch 4     | Epoch Time: 0m 7s 
        | Train loss    0.676| Train acc    0.771 | Valid loss    0.658 | Valid acc    0.773 | + 


KeyboardInterrupt: 

> Can follow all the same steps and just change Conv2D model for Conv1D model. Dosnt change the accuracy much.

### CNN with Conv1D

In [ ]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
def predict_cnn(model, field, text):
    model.eval()
    text = field.preprocess(text)
    text = field.process([text])
    x = torch.tensor(text)
    # x = autograd.Variable(x)
    x = x.cuda()
    logits = model(x)
    y_pred = torch.max(logits, dim=-1)[1]
    y_pred = y_pred.item()
    return y_pred

In [ ]:
cnn_preds = [[test_preprocessed.id[i],predict_cnn(CNN, TEXTcnn, test_preprocessed.AllCombined[i])] for i in range(len(test))]

In [ ]:
kaggle_cnn = pd.DataFrame(data=cnn_preds,columns=['id','label'])

In [ ]:
kaggle_cnn.to_csv("kaggle_cnn_best.csv",index=None)